In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
import time

In [6]:
# Configuración de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# Transformaciones para normalizar imágenes (Requeridas por ResNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar imágenes
    transforms.ToTensor(),  # Convertir a tensores
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización de ResNet
])

In [10]:
# Rutas a los conjuntos de entrenamiento y validación
train_dir = "./Data/train"
valid_dir = "./Data/valid"

In [11]:
# Cargar imágenes desde carpetas (PyTorch usa estructura basada en subcarpetas para cada clase)
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)

In [12]:
# Definir DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [14]:
# Verificar categorías detectadas
print("Clases detectadas:", train_dataset.classes)

Clases detectadas: ['freshripe', 'freshunripe', 'overripe', 'ripe', 'rotten', 'unripe']


In [15]:
# Cargar modelo ResNet-18 preentrenado
num_classes = 6  # Tenemos 6 categorías
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modificar capa final para 6 clases
model = model.to(device)

c:\Users\Raxie\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Raxie\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Raxie/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 85.1MB/s]


In [16]:
# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Entrenamiento del modelo
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    start_time = time.time()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Acc: {acc:.2f}%, Time: {time.time() - start_time:.2f}s")

In [ ]:
# Evaluación del modelo en validación
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
val_acc = 100 * correct / total
print(f"Accuracy en validación: {val_acc:.2f}%")

In [ ]:
# Guardar el modelo entrenado
torch.save(model.state_dict(), "resnet6_classes.pth")
print("Modelo guardado como 'resnet6_classes.pth'")